In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
CLIENT_ID = 'PBXJVIR2BNFY1DBCDEBZAGKJ2F0XKORKZ1SUICVFYQIHM1AI' # your Foursquare ID
CLIENT_SECRET = 'TZ5LPDVYAEDZOTY3LTC0FOPXLAZVWNAKLVPHKZE0VS1OSLCZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PBXJVIR2BNFY1DBCDEBZAGKJ2F0XKORKZ1SUICVFYQIHM1AI
CLIENT_SECRET:TZ5LPDVYAEDZOTY3LTC0FOPXLAZVWNAKLVPHKZE0VS1OSLCZ


In [2]:
address = '190 Elizabeth St, Brisbane City, QLD 4000, Australia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-27.46915765 153.026801032079


In [3]:
search_query = 'Phones'
radius = 1000
print(search_query + ' .... OK!')

Phones .... OK!


In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PBXJVIR2BNFY1DBCDEBZAGKJ2F0XKORKZ1SUICVFYQIHM1AI&client_secret=TZ5LPDVYAEDZOTY3LTC0FOPXLAZVWNAKLVPHKZE0VS1OSLCZ&ll=-27.46915765,153.026801032079&v=20180604&query=Phones&radius=1000&limit=30'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8dcd751ed219672c97d79f'},
 'response': {'venues': [{'id': '4b5d0f12f964a520c85029e3',
    'name': 'Allphones',
    'location': {'address': 'Shop 118 Level E, Myer Centre',
     'crossStreet': '91 Queen St.',
     'lat': -27.470789680000003,
     'lng': 153.024964,
     'labeledLatLngs': [{'label': 'display',
       'lat': -27.470789680000003,
       'lng': 153.024964}],
     'distance': 256,
     'postalCode': '4000',
     'cc': 'AU',
     'city': 'Brisbane',
     'state': 'QLD',
     'country': 'Australia',
     'formattedAddress': ['Shop 118 Level E, Myer Centre (91 Queen St.)',
      'Brisbane QLD 4000',
      'Australia']},
    'categories': [{'id': '4bf58dd8d48988d122951735',
      'name': 'Electronics Store',
      'pluralName': 'Electronics Stores',
      'shortName': 'Electronics',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/technology_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-155

In [11]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",False,4b5d0f12f964a520c85029e3,"Shop 118 Level E, Myer Centre",AU,Brisbane,Australia,91 Queen St.,256,"[Shop 118 Level E, Myer Centre (91 Queen St.),...","[{'label': 'display', 'lat': -27.4707896800000...",-27.47079,153.024964,4000,QLD,Allphones,v-1552797045


In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Allphones,Electronics Store,"Shop 118 Level E, Myer Centre",AU,Brisbane,Australia,91 Queen St.,256,"[Shop 118 Level E, Myer Centre (91 Queen St.),...","[{'label': 'display', 'lat': -27.4707896800000...",-27.47079,153.024964,4000,QLD,4b5d0f12f964a520c85029e3


In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map